# About OCR approach1:
Through ocr1.py script we are targeting to train a small Convolutional Neurl Network (CNN)  with the data we generated using random_string_data_gen.py. Network should be able to recognize the random string in a given image and provide it as ouput.  
In the first phase we will be testing it using generated data itself whereas later we try to crop some image from our screen meeting dataset contrain and see how well network works.  
Link to blog: https://medium.com/@vijendra1125/ocr-part-2-ocr-using-cnn-f43f0cee8016

## Load Libraries

In [ ]:
import os
from datetime import datetime as dt
import string
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

from ocr1_functions import *

# set ro print numpy array without truncation
np.set_printoptions(threshold=sys.maxsize)

bold = '\033[1m'
end = '\033[0m'

### Parameters

In [ ]:
## paths ##
# path to folder where data (tfrecord) files has been stored
folder_path = "../data"
# path to load checkpoint
checkpoint_restore = "../cp/ocr1_3to8"
# path to save checkpoint 
checkpoint_save = "../cp/ocr1_3to8"

## data realted params ##
# train and test file paths
filenames = os.listdir(folder_path)
train_file_paths = []
test_file_paths = []
for filename in filenames:
    if "train" in filename:
        train_file_paths.append(os.path.join(folder_path, filename))
    elif "test" in filename:
        test_file_paths.append(os.path.join(folder_path, filename))
# total number of data in each train tfrecord
# data_per_train_file = 8192
total number of data in each test tfrecord
data_per_test_file = 2048
# image size
image_size = [32,256,1]
# total numember of classes
class_count = 63
# string length (including whitespace)
string_length = 16

## training  setup related params ##
# restore from given checkpoint
restore = False
# dropout for each layer (1 means no drop)
dropout = [1, 1, 1, 1]
# weight decay
wd = 0.000
# learning rafe
lr = 0.01
# batch size
# batch_size = 32
batch_size = 1
# total number of epochs
epochs = 5
# after every x epoch decrease learning rate by factor of y (var_lr = [x, y])
var_lr=[None,None]
# parameters related to reading tfrecord
num_of_threads=16
min_after_dequeue=5000
capacity=min_after_dequeue+(num_of_threads+1)*batch_size

### Train

In [ ]:
# data count
train_data_count = data_per_train_file * len(train_file_paths)
test_data_count = data_per_test_file * len(test_file_paths)
# steps 
train_step = train_data_count//batch_size
test_step = test_data_count//batch_size 
# build graph
with tf.Graph().as_default():
    # train graph
	x_train, y_train = minibatch(batch_size, train_file_paths, image_size, string_length, class_count)     
	logit_train = inference(x_train, class_count, dropout = dropout, wd = wd)
	cost = multi_loss(logit_train, y_train, batch_size, string_length)
	update=parameter_update(cost,lr)	
	accuracy_train = accuracy_calc(logit_train, y_train)
    # test graph
	x_test, y_test = minibatch(batch_size, test_file_paths, image_size, string_length, class_count)
	logit_test = inference(x_test, class_count)
	accuracy_test = accuracy_calc(logit_test, y_test)  
	saver = tf.train.Saver()   
    # start session
	with tf.Session() as sess:
    	# initialize the variables
		sess.run(tf.global_variables_initializer())
		sess.run(tf.local_variables_initializer())
		coord = tf.train.Coordinator()
		threads = tf.train.start_queue_runners(coord=coord)      
    	# restore the variables
		if restore == True:
			loader = tf.train.import_meta_graph(checkpoint_restore +'.meta')
			loader.restore(sess, checkpoint_restore)       
		# train for given number of epochs
		for e in range(epochs): 
			print(bold + "\nepoch:" + end, e)
			train_epoch_cost = 0
			train_epoch_acc = 0
			test_epoch_acc = 0        
            # train for given number of steps in one epoch
			for s in range(train_step):
				_,train_batch_cost = sess.run([update, cost])	          
				if s % (train_step//2) == 0 and s != 0:
					print('~', end = '')
				elif(s == (train_step) - 1):
					print('')            
				train_epoch_cost += train_batch_cost/(train_step)	          
			print(bold + "epoch_cost: " + end,train_epoch_cost)       
            # calculate accuracy of training set
			for i in range(train_step//5):
				train_epoch_acc = sess.run(accuracy_train)
				train_epoch_acc += train_epoch_acc/(train_step)        
			print(bold + "train epoch accuracy: " + end,train_epoch_acc, "\n")        
            # calculate accuracy on test set
			for i in range(test_step):
				test_epoch_acc = sess.run(accuracy_test)
				test_epoch_acc += test_epoch_acc/test_step    
			print(bold + "test epoch accuracy: " + end, test_epoch_acc, "\n")       
            # after every x epoch decrease learning rate by factor of y (var_lr = [x, y])
			if var_lr[0] != None:
				if e%var_lr[0] == 0:
					learning_rate = learning_rate/var_lr[1]     
        #save all the variables		
		save_path = saver.save(sess, checkpoint_save)	
		coord.request_stop()
		coord.join(threads)
		print("---training over---")

### Evaluate

In [ ]:
steps=((test_data_count))//batch_size
accu=0
x_test, y_test = minibatch(batch_size, test_file_paths, image_size, string_length, class_count)
logit_test = inference(x_test, class_count)
accuracy_test = accuracy_calc(logit_test, y_test)  
init=tf.global_variables_initializer()
saver=tf.train.Saver()
with tf.Session() as sess:
	sess.run(init)
	coord = tf.train.Coordinator()
	threads = tf.train.start_queue_runners(coord=coord)
	saver.restore(sess,checkpoint_restore)
	for s in range(steps):
		acc=sess.run(accuracy_test)
		accu+=acc/steps
	print("test set accuracy: ",acc)
	coord.request_stop()
	coord.join(threads)

In [ ]:
all_chr = list(string.ascii_letters) + list(string.digits) + list(' ')
x_check, y_check=minibatch(batch_size, test_file_paths, image_size, string_length, class_count) 
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(tf.local_variables_initializer())
  coord = tf.train.Coordinator()
  threads = tf.train.start_queue_runners(coord=coord) 
  x_c = sess.run(x_check)  
  eval_vizualization(x_c[1:5])
  coord.request_stop()
  coord.join(threads)